In [ ]:
# !pip install requests beautifulsoup4 markdownify

This script scrapes agricultural guides for five specific crops from AgriCash, automatically visiting their detailed pages to convert HTML content into clean Markdown. It then compiles the plant names, source URLs, and formatted text into a structured JSONL dataset optimized for training Q&A models.##

In [3]:
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md
import json
import time

# إعدادات المتصفح لتجنب الحظر
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_soup(url):
    """دالة مساعدة لجلب محتوى الصفحة وتحويله لـ Soup"""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def scrape_plant_right():
    base_url = "https://agricash.app/plant-right/"
    output_file = "plant_right.jsonl"
    
    print("1. جاري الدخول للصفحة الرئيسية...")
    soup = get_soup(base_url)
    if not soup:
        return

    
    target_crops = ["الذرة", "الشمر", "الريحان", "البصل", "بنجر"]
    extracted_data = []

    all_headings = soup.find_all(['h2', 'h3', 'h4', 'h5'])
    
    for heading in all_headings:
        text = heading.get_text(strip=True)
        
        # التأكد أن العنوان هو أحد المحاصيل المستهدفة
        if any(crop in text for crop in target_crops):
            print(f"-- تم العثور على محصول: {text}")
            
            plant_name = text
            
        
            card = heading.find_parent() 
            
            summary = ""
            summary_tag = heading.find_next_sibling('p')
            if summary_tag:
                summary = summary_tag.get_text(strip=True)
            
            # البحث عن رابط التفاصيل "إضغط هنا" أو الرابط المباشر
            link_tag = heading.find_next('a', href=True)
            detailed_url = link_tag['href'] if link_tag else None
            
            if detailed_url:
                # معالجة الرابط إذا كان نسبيًا (Relative URL)
                if not detailed_url.startswith('http'):
                    detailed_url = f"https://agricash.app{detailed_url}"
                
                print(f"   جاري سحب التفاصيل من: {detailed_url}")
                
                # --- الخطوة ب: سحب الصفحة التفصيلية ---
                detailed_markdown = get_detailed_content(detailed_url)
                
                # --- الخطوة ج: تنسيق المخرج ---
                # التنسيق المطلوب: # [Crop Name][Summary]---[Content]
                full_markdown = f"# {plant_name}\n{summary}\n---\n{detailed_markdown}"
                
                record = {
                    "plant_name": plant_name,
                    "source_url": detailed_url,
                    "markdown_content": full_markdown
                }
                
                extracted_data.append(record)
                
                # انتظار بسيط لتجنب الضغط على السيرفر
                time.sleep(1)

    # حفظ البيانات في ملف JSONL
    print(f"3. جاري الحفظ في {output_file}...")
    with open(output_file, 'w', encoding='utf-8') as f:
        for entry in extracted_data:
            json.dump(entry, f, ensure_ascii=False)
            f.write('\n')
    
    print("تمت المهمة بنجاح!")

def get_detailed_content(url):
    """دالة لسحب محتوى الصفحة التفصيلية وتنظيفه وتحويله لمارك داون"""
    soup = get_soup(url)
    if not soup:
        return ""
    
    # محاولة تحديد المحتوى الأساسي فقط (تجاهل الهيدر والفوتر)
    # عادة ما يكون المحتوى داخل <article> أو <main> أو div class="content"
    content_div = soup.find('article') or soup.find('main') or soup.find('div', class_='entry-content')
    
    if not content_div:
        # إذا فشل التحديد، خذ البودي كله (خيار أخير)
        content_div = soup.body

    # تنظيف العناصر غير المرغوب فيها قبل التحويل
    for unwanted in content_div.find_all(['script', 'style', 'nav', 'footer', 'header', 'form']):
        unwanted.decompose()

    # تحويل HTML إلى Markdown
    # heading_style='ATX' يجعل العناوين تبدأ بـ # بدلاً من خط تحتها
    markdown_text = md(str(content_div), heading_style="ATX")
    
    # تنظيف إضافي للنص الناتج
    lines = []
    for line in markdown_text.split('\n'):
        line = line.strip()
        # حذف جملة "إضغط هنا" أو الروابط الفارغة
        if "إضغط هنا" in line or line == "":
            continue
        lines.append(line)
        
    return "\n".join(lines)

if __name__ == "__main__":
    scrape_plant_right()

1. جاري الدخول للصفحة الرئيسية...
-- تم العثور على محصول: الذرة الشامية والرفعية
   جاري سحب التفاصيل من: https://agricash.app/corn-and-sorghum/
-- تم العثور على محصول: الشمر
   جاري سحب التفاصيل من: https://agricash.app/%D8%AE%D8%AF%D9%85%D8%A9%20%D9%88%20%D8%B2%D8%B1%D8%A7%D8%B9%D8%A9%20%D8%A7%D9%84%D8%B4%D9%85%D8%B1/
Error fetching https://agricash.app/%D8%AE%D8%AF%D9%85%D8%A9%20%D9%88%20%D8%B2%D8%B1%D8%A7%D8%B9%D8%A9%20%D8%A7%D9%84%D8%B4%D9%85%D8%B1/: 404 Client Error: Not Found for url: https://agricash.app/%D8%AE%D8%AF%D9%85%D8%A9%20%D9%88%20%D8%B2%D8%B1%D8%A7%D8%B9%D8%A9%20%D8%A7%D9%84%D8%B4%D9%85%D8%B1/
-- تم العثور على محصول: الريحان
   جاري سحب التفاصيل من: https://agricash.app/%d8%ae%d8%af%d9%85%d8%a9-%d9%88%d8%b2%d8%b1%d8%b9%d8%a9-%d8%a7%d9%84%d8%b1%d9%8a%d8%ad%d8%a7%d9%86/
-- تم العثور على محصول: البصل
   جاري سحب التفاصيل من: https://agricash.app/onion-cultivation/
-- تم العثور على محصول: بنجر السكر
   جاري سحب التفاصيل من: https://agricash.app/sugar-beet/?
3. جاري الحفظ 

In [6]:
import json
import re

# Load JSONL
with open('plant_right.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

def remove_images(md_content):
    """
    Remove all Markdown image tags from the content
    """
    # Regex to match Markdown images ![alt](url)
    md_no_images = re.sub(r'!\[.*?\]\(.*?\)', '', md_content)
    return md_no_images

# Apply cleaning
for item in data:
    item['markdown_content'] = remove_images(item['markdown_content'])

# Save cleaned JSONL
with open('plant_right.jsonl', 'w', encoding='utf-8') as f:
    for item in data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"Removed images from {len(data)} items and saved to 'plant_right_cleaned_no_images.jsonl'")


Removed images from 5 items and saved to 'plant_right_cleaned_no_images.jsonl'


In [ ]:
# import json
# import os

# # Load JSONL
# with open('plant_right_cleaned_no_images.jsonl', 'r', encoding='utf-8') as f:
#     data = [json.loads(line) for line in f]


# with open('plant_right_first_item.md', 'w', encoding='utf-8') as md_file:
#     # \n in string will create real line breaks in the .md file
#     md_file.write(data[0]['markdown_content'])
